In [77]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

#Import Plotly
import plotly.express as px
import plotly.graph_objs as go

#import sklearn
from sklearn.preprocessing import MinMaxScaler

In [78]:
combine_df = pd.read_csv('resources/combine_df.csv',index_col=None)

# Fix incorrect data from source
combine_df.loc[(combine_df['Name']=='Spencer Brown') & (combine_df['College']=='Alabama-Birmingham'), 'Bench Press'] = 16
combine_df.loc[(combine_df['Name']=='Jacoby Ford') & (combine_df['College']=='Clemson'), '40 Yard'] = 4.28
combine_df = combine_df.loc[(combine_df['Name'] != 'Trindon Holliday')]

combine_df.BMI = combine_df.BMI.round(1)

combine_df.to_csv('resources/combine_df.csv', index=False)

combine_df.head()



,Year,Name,College,POS,Height (in),Weight (lbs),BMI,40 Yard,Bench Press,Vert Leap (in),Broad Jump (in),Shuttle,3Cone
0,1987,Mike Adams,Arizona State,CB,69.8,198.0,28.6,4.42,13.0,32.0,118.0,4.60,NaN
1,1987,John Adickes,Baylor,C,74.8,266.0,33.4,4.97,25.0,26.5,103.0,4.60,NaN
2,1987,Tommy Agee,Auburn,FB,71.8,217.0,29.6,NaN,15.0,NaN,NaN,NaN,NaN
3,1987,David Alexander,Tulsa (OK),C,75.0,279.0,34.9,5.13,22.0,27.5,105.0,4.33,NaN
4,1987,Lyneal Alston,Southern Mississippi,WR,72.1,202.0,27.3,4.64,7.0,32.0,114.0,4.52,NaN


In [69]:
#Get the averages for all stats and make it presentable in a summarized DF.
combine_df_avg = pd.DataFrame(
    [{"Avg. Height (in)":combine_df['Height (in)'].mean(),
      "Avg. Weight (lbs)":combine_df['Weight (lbs)'].mean(),
      "Avg. BMI":combine_df['BMI'].mean(),
      "Avg. 40 Yard":combine_df['40 Yard'].mean(), 
      "Avg. Vert Leap (in)":combine_df['Vert Leap (in)'].mean(),
      "Avg. Broad Jump (in)":combine_df['Broad Jump (in)'].mean(),
      "Shuttle":combine_df['Shuttle'].mean(),
      "3Cone":combine_df['3Cone'].mean(),
      "Bench Press":combine_df['Bench Press'].mean(),
     }])


combine_df_avg

,Avg. Height (in),Avg. Weight (lbs),Avg. BMI,Avg. 40 Yard,Avg. Vert Leap (in),Avg. Broad Jump (in),Shuttle,3Cone,Bench Press
0,73.747822,240.121155,30.852765,4.811814,32.272046,113.311822,4.41903,7.310605,19.806189


In [70]:
# Get records for all combine data, 

record_stats_df = pd.DataFrame(
    [{"Max Height (in)":combine_df['Height (in)'].max(),
      "Max Weight (lbs)":combine_df['Weight (lbs)'].max(),
      "Max BMI":combine_df["BMI"].max(),
      "Min 40 Yard":combine_df['40 Yard'].min(),
      "Max Bench Press":combine_df['Bench Press'].max(), 
      "Max Vert Leap (in)":combine_df['Vert Leap (in)'].max(),
      "Max Broad Jump (in)":combine_df['Broad Jump (in)'].max(),
      "Min Shuttle":combine_df['Shuttle'].min(),
      "Min 3Cone":combine_df['3Cone'].min(),
    }])

# Transpose the dataframe:
record_stats_df_trans = record_stats_df.T
record_stats_df_trans.reset_index(inplace=True)
record_stats_df_trans.columns = ['Measure', 'Values']
record_stats_df_trans.Values = record_stats_df_trans.Values.round(2)
record_stats_df_trans

,Measure,Values
0,Max Height (in),82.40
1,Max Weight (lbs),387.00
2,Max BMI,47.70
3,Min 40 Yard,4.21
4,Max Bench Press,51.00
5,Max Vert Leap (in),46.50
6,Max Broad Jump (in),147.00
7,Min Shuttle,3.73
8,Min 3Cone,6.27


In [71]:
# Get all rows wth record holders

records_df = pd.DataFrame()
val_columns = combine_df.columns[4:]

for item in val_columns:
    try:
        if item == '40 Yard' or item =='Shuttle' or item =='3Cone':
            records_df = records_df.append(combine_df.loc[combine_df[item] == combine_df[item].min()])
        else:
            records_df = records_df.append(combine_df.loc[combine_df[item] == combine_df[item].max()])
    except:
        print('not a number column')

records_df.reset_index(drop=True,inplace=True)
records_df.drop(records_df.iloc[:,4:],inplace=True,axis=1)

In [72]:
# Combine transposed data with record holder data
records_players_df = records_df.merge(record_stats_df_trans,left_index=True,right_index=True)

records_players_df.to_csv('resources/records_players_df.csv')

records_players_df

,Year,Name,College,POS,Measure,Values
0,1996,Mike Rockwood,Nevada Las Vegas,OT,Max Height (in),82.40
1,1997,Jamie Nails,Florida A&M,OT,Max Weight (lbs),387.00
2,1993,Eddie Smith,Texas Southern,DT,Max BMI,47.70
3,2010,Trindon Holliday,Louisiana State,WR,Min 40 Yard,4.21
4,1999,Justin Ernest,Eastern Kentucky,DT,Max Bench Press,51.00
5,2021,Josh Imatorbhebhe,Illinois,WR,Max Vert Leap (in),46.50
6,2015,Byron Jones,Connecticut,CB,Max Broad Jump (in),147.00
7,2001,Kevin Kasper,Iowa,WR,Min Shuttle,3.73
8,2007,Mike Richardson,Notre Dame,CB,Min 3Cone,6.27


In [73]:
scaler = MinMaxScaler()

df_values = combine_df.drop(['Name','Year','College','POS'], axis=1)
df_info = combine_df.drop(['Height (in)', 'Weight (lbs)', 'BMI', '40 Yard', 'Bench Press', 'Vert Leap (in)', 'Broad Jump (in)', 'Shuttle', '3Cone'],axis=1)


df_scaled = scaler.fit_transform(df_values.to_numpy())
df_scaled = pd.DataFrame(df_scaled, columns=[
  'Height (in)', 'Weight (lbs)', 'BMI', '40 Yard', 'Bench Press', 'Vert Leap (in)', 'Broad Jump (in)', 'Shuttle', '3Cone'])
 
print("Scaled Dataset Using MinMaxScaler")

combine_df_scaled = df_info.merge(df_scaled,left_index=True, right_index=True)

# Reverse order for speed measure so that faster times are the max
for col in ['40 Yard', 'Shuttle', '3Cone']:
    combine_df_scaled[col] = 1 - combine_df_scaled[col]

combine_df_scaled.to_csv("resources/combine_df_scaled.csv", index=False)

combine_df_scaled.head()

Scaled Dataset Using MinMaxScaler


,Year,Name,College,POS,Height (in),Weight (lbs),BMI,40 Yard,Bench Press,Vert Leap (in),Broad Jump (in),Shuttle,3Cone
0,1987,Mike Adams,Arizona State,CB,0.397129,0.228571,0.276515,0.890052,0.24,0.500000,0.792857,0.749280,NaN
1,1987,John Adickes,Baylor,C,0.636364,0.506122,0.458333,0.602094,0.48,0.310345,0.685714,0.749280,NaN
2,1987,Tommy Agee,Auburn,FB,0.492823,0.306122,0.314394,NaN,0.28,NaN,NaN,NaN,NaN
3,1987,David Alexander,Tulsa (OK),C,0.645933,0.559184,0.515152,0.518325,0.42,0.344828,0.700000,0.827089,NaN
4,1987,Lyneal Alston,Southern Mississippi,WR,0.507177,0.244898,0.227273,0.774869,0.12,0.500000,0.764286,0.772334,NaN


In [74]:
combine_df.POS.value_counts()

WR     1836
CB     1406
RB     1219
DE     1060
OT     1044
DT     1024
OLB    1024
OG      847
TE      789
QB      755
ILB     555
FS      535
SS      485
C       436
FB      317
P       163
K       145
LB       91
S        53
LS       26
EDG      22
OL       10
DL        4
DB        3
NT        1
Name: POS, dtype: int64

In [75]:
combine_df.loc[combine_df['POS'] == 'DB']

,Year,Name,College,POS,Height (in),Weight (lbs),BMI,40 Yard,Bench Press,Vert Leap (in),Broad Jump (in),Shuttle,3Cone
10604,2017,Brian Allen,Utah,DB,74.88,215.0,27.0,4.48,15.0,34.5,117.0,4.34,6.64
10682,2017,Chuck Clark,Virginia Tech,DB,71.63,208.0,28.5,4.54,16.0,34.0,122.0,4.07,6.85
10793,2017,Shaquill Griffin,Central Florida,DB,71.75,194.0,26.5,4.38,17.0,38.5,132.0,4.14,6.87
